In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.svm import SVC
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from google.colab import drive
drive.mount('/content/drive',force_remount=False)
dataset = '/content/drive/My Drive/ConcatFeat.csv'
#df = pd.read_csv("/Concatenated.csv")
df = pd.read_csv(dataset)
X = df.drop(labels='Label',axis=1)
Y = df['Label']
df.shape
#
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
!nvcc --version
!pip install thundersvm

Mounted at /content/drive
--2021-06-18 17:44:29--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2021-06-18 17:44:29--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?urPRxdxf_ybfJDydQRVY_AHV3q8JdTE1hX9vFpM3hzzkEDcu-JrdPrbuxN7EDy_lZ4iTERF

In [28]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from thundersvm import SVC
import statistics
kf = StratifiedKFold(n_splits=10, shuffle= True, random_state=100)
splits = kf.split(X,Y)
#,  gamma, , ----
win = 16
parm = {'C': [0.001,0.01, 0.1, 1, 10, 100],
        'gamma':[1, 0.1, 0.01]}
acc = []
sen = []
spe = []
pre = []
auc = []
for train_index, val_index in splits:
  Y_train = Y[train_index]
  Y_test = Y[val_index]
  X_train = X.values[train_index,]
  X_test = X.values[val_index,]
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  clf = GridSearchCV(SVC(kernel='rbf',probability = True), parm, cv=10, verbose=0)
  clf.fit(X_train,Y_train)
  svm = clf.best_estimator_
  Y_predict = np.array(svm.predict(X_test),dtype=int) # np.array included
  prob = svm.predict_proba(X_test)
  # MAjority voting
  L = len(Y_test)
  Y_test = np.array(Y_test)
  Y_test = Y_test[np.arange(0,L-1,16)]
  LL = np.int(L/win)
  prob1 = np.zeros(shape=(LL,2))
  Pred1 = []
  for i in range(LL):
    prob = np.array(prob)
    ini = i*win; fin = win*(i+1)
    prob1[i,:] = np.mean(prob[ini:fin,:],axis=0) 
    Pred1.append(statistics.mode(Y_predict[ini:fin]))
  # majority voting end
  C = confusion_matrix(Y_test,Pred1)
  print(C)
  TP = C[1,1]
  FN= C[1,0]
  FP= C[0,1]
  TN= C[0,0]
  sen.append(TP /(TP + FN))
  spe.append(TN /(TN + FP))
  pre.append(TP /(TP + FP))
  prob = np.array(prob)
  auc.append(roc_auc_score(Y_test,prob1[:,0]))
  acc.append((TN+TP)/(TP+TN+FP+FN))

[[32  0]
 [ 0 32]]
[1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[[32  0]
 [ 0 32]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
results = pd.DataFrame(np.matrix.transpose(np.matrix([sen,spe,pre,acc,auc])),
             columns=['Sensitvity','Specificity','Precision','Accuracy','AUCROC'])
results.to_csv("SVMResults.csv")
results

,Sensitvity,Specificity,Precision,Accuracy,AUCROC
0,1.0,1.0,1.0,1.0,0.5
1,1.0,1.0,1.0,1.0,0.5
2,1.0,1.0,1.0,1.0,0.5
3,1.0,1.0,1.0,1.0,0.5
4,1.0,1.0,1.0,1.0,0.5
5,1.0,1.0,1.0,1.0,0.5
6,1.0,1.0,1.0,1.0,0.5
7,1.0,1.0,1.0,1.0,0.5
8,1.0,1.0,1.0,1.0,0.5
9,1.0,1.0,1.0,1.0,0.5
